In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from dotenv import dotenv_values
env_variables = dotenv_values('Ignore.env')

In [ ]:
import mysql.connector

In [ ]:
#Connecting to SQL database
mydb = mysql.connector.connect(
  host=env_variables['HOST'],
  user=env_variables['USER'],
  password=env_variables['PASSWORD'],
  port=env_variables['PORT']
)

print(mydb)

In [ ]:
#Pulling data from sql database
cursor = mydb.cursor()
cursor.execute("USE ifcom_prod")
cursor.execute("SELECT c.IDCyc, c.WhenStartPickup, c.IDMemb, c.WhenReg, c.Addr1, c.Addr2, c.City, c.St, c.Zip, c.CDLocLast, l.NumLogins FROM (SELECT IDCyc, WhenStartPickup, IDMemb, WhenReg, Addr1, Addr2, City, St, Zip, CDLocLast FROM Cyc CROSS JOIN Memb) c LEFT JOIN (SELECT IDMemb, COUNT(IDMemb) AS NumLogins FROM Login GROUP BY IDMemb) l ON c.IDMemb = l.IDMemb")

In [ ]:
results = cursor.fetchall()
column_names = [description[0] for description in cursor.description]
member_df = pd.DataFrame(results, columns=column_names)

In [ ]:
print(f"Number of rows returned: {len(results)}")
print(f"Column names: {[description[0] for description in cursor.description]}")

In [ ]:
cursor.close()
mydb.close()

In [ ]:
mydb = mysql.connector.connect(
  host=env_variables['HOST'],
  user=env_variables['USER'],
  password=env_variables['PASSWORD'],
  port=env_variables['PORT']
)
cursor = mydb.cursor()
cursor.execute("USE ifcom_prod")
cursor.execute("SELECT IDMemb, Addr1,Addr2, City,St,Zip,CDLocLast, CkFounder, HowHear, CDRegMemb FROM Memb")
results = cursor.fetchall()
column_names = [description[0] for description in cursor.description]
member_names_df = pd.DataFrame(results, columns=column_names)

In [ ]:
cursor.close()
mydb.close()

In [ ]:
initial_df = pd.read_csv('producer-sales-report-3.csv')

In [ ]:
initial_df.head(-5)

In [ ]:
member_df.head()

In [ ]:
#dropping columns that aren't needed
member_names_df.drop(["Addr1","Addr2", "City", "St","Zip", "CDLocLast"], axis=1, inplace = True)

In [ ]:
member_names_df.head()

In [ ]:
#Comnbining Membership info here so that there is a row for every customer for every ordering period.
#It is now ready to merge onto the sales data.
customer_df = member_df.merge(member_names_df, how='left', on = ["IDMemb"])

In [ ]:
#Pulled data in the middle of a cycle. Deleting all data in cycles after 334 so I'm only evaluating complete cycles.
customer_df = customer_df[customer_df['IDCyc']<335]
customer_df.head()

In [ ]:
initial_df.columns

In [ ]:
#Dropping more columns that aren't needed.
sales_df = initial_df.drop(['IDVty','FeeCoop','TaxSale','Producer','CustEmail','CustPhone','NameProduct','CustomerName','QtyDeliv','FeeCoopForgiv'],axis=1)

In [ ]:
#Don't want to look at sales onsite. Only web sales
sales_df=sales_df[sales_df['saleSource']!='onsite']

In [ ]:
#Data was pulled in the middle of a cycle. Getting rid of everything after cycle id 334.
sales_df = sales_df[sales_df['IDCyc'] <= 334]

In [ ]:
#By making a pivot table here, I am aggregating each customer's purchases by category by week purchased. I will then be able to add each
# category as a feature to the dataframe with how much each customer spent on each category each ordering period.
cats_to_cols = sales_df.pivot_table(index=['IDCyc','IDMemb'], values = 'SaleNom',columns = 'NameCat')

In [ ]:
cats_to_cols

In [ ]:
cats_to_cols.fillna(0,inplace=True)

In [ ]:
cats_to_cols

In [ ]:
#Making another pivot table that adds the total sales for each customer during each ordering period. I will merge this back in to the main dataframe later.
customer_sales_by_cycle_df = pd.pivot_table(sales_df, values='SaleNom', index =['IDMemb','IDCyc'], aggfunc=np.sum)

In [ ]:
customer_sales_by_cycle_df.head()

In [ ]:
sales_df.head()

In [ ]:
#Merging data backinto the main dataframe.
combined_df = customer_df.merge(customer_sales_by_cycle_df, how = 'left', on = ['IDMemb','IDCyc'])

In [ ]:
combined_df.head()

In [ ]:
#new_df has all members in it, even those who have never ordered. Need to think about removing people if they have never ordered.

new_df = combined_df.merge(cats_to_cols, how = 'left', on = ['IDMemb','IDCyc'])

In [ ]:
#Checking to see how many unique customer ID's we have, and of those how many have ordered over the last 2 years.
total_members=len(new_df['IDMemb'].unique())
customers_ordering=len(new_df[new_df['SaleNom']>0]['IDMemb'].unique())
print(f"The IFC has {total_members} total members and {customers_ordering} of them have ordered in the last two years.")

In [ ]:
#Preparing for calculating distance to pick up. Need to clean up the address info.
new_df['St'].unique()

In [ ]:
#Replacing typos when peopel typed in their state.
new_df['St'].replace('IO','IA',inplace = True)
new_df['St'].replace('Ia','IA',inplace = True)
new_df['St'].replace('ia','IA',inplace = True)
new_df['St'].replace('22','IA',inplace = True)
new_df['St'].replace('I','IA',inplace = True)

In [ ]:
new_df['St'].unique()

In [ ]:
#These are the only people who have ordered who are not from Iowa.
new_df[(new_df['St']!='IA') & new_df['SaleNom']!=0]

In [ ]:
#Dropping all members not from Iowa who have never ordered. This should make it easier to compute addresses and distances in the next step.
new_df=new_df.drop(new_df[new_df['St'].isin(['CO', 'MN', 'SD', 'TN', 'AR', 'MO', 'FL', 'PO', 'AK', 'NE',
       'NY', 'IL', 'AZ', 'SO', 'NV', 'WI', 'PA', 'KS', 'TX', 'AL',
       'WA', 'XT', 'XE', 'OR', 'IS', 'MI', 'GA', 'AI', 'RI', '14',
       'MD', 'NJ'])].index)

In [ ]:
new_df.columns

In [ ]:
# As I got further along, apartment numbers did not work when I tried to find their geolocation. This code will remove
# the apartment numbers from the addresses
new_df['Addr1'] = new_df['Addr1'].str.split('#').str[0]
new_df['full_address'] = new_df.Addr1 + ',' + new_df.City + ',' + new_df.St + ',' + new_df.Zip

In [ ]:
#These are the locations where members can pick up their orders.
sites = {'FRAN': (41.61144913179401, -93.68691528533463),
"ANK" :(41.72639503101638, -93.58202826441813),
"HOME" :('nan'),
"WDM" :(41.58160655660526, -93.83719844907307),
"IND" :(41.364019620478196, -93.56060397936696),
"PAN" :(41.653588359660176, -94.32091117790932),
"HILL" :(41.60142613460194, -93.5138259018194),
}

In [ ]:
#Converting sites to a dataframe
sites_df = pd.DataFrame(sites)
sites_df = sites_df.T
sites_df.rename({0:'Pickup_lat',1:'Pickup_long'},axis = 1,inplace = True)
sites_df.reset_index(inplace=True)
sites_df.rename({'index': 'CDLocLast'},axis=1,inplace = True)
sites_df.head()

In [ ]:
#shrinking my dataframe to only get every member once time. This way I only have to calculate each distance one time.
geocoding_df = new_df[new_df['IDCyc'] == new_df['IDCyc'].max()]

In [ ]:
geocoding_df.head()

In [ ]:
api_key = env_variables['API_KEY']

In [ ]:
from geopy.geocoders import GoogleV3
import requests
geolocator = GoogleV3(api_key=api_key)

In [ ]:
#defining a function to calculate the lat and long of each address.
def geocode_address(address):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={api_key}"
    response = requests.get(url)
    data = response.json()
    if data['status'] == 'OK':
        lat = data['results'][0]['geometry']['location']['lat']
        lng = data['results'][0]['geometry']['location']['lng']
        return lat, lng
    else:
        return None, None


In [ ]:
#getting latitude and longitude coordinates for each address in dataframe.
geocoding_df[['latitude', 'longitude']] = geocoding_df['full_address'].apply(geocode_address).apply(pd.Series)

In [ ]:
geocoding_df.head()

In [ ]:
#Adding the sites dataframe so we can calulate distance between pickup location and each customer's address.
geocoding_df = geocoding_df.merge(sites_df, how = 'left', on ='CDLocLast')

In [ ]:
geocoding_df=geocoding_df.astype({'latitude':'float','longitude':'float', 'Pickup_lat':'float', 'Pickup_long':'float'})

In [ ]:
geocoding_df[geocoding_df['latitude'].isna()]

In [ ]:
geocoding_df=geocoding_df[(geocoding_df['IDMemb']!=1347) & (geocoding_df['IDMemb']!=1351)]

In [ ]:
import geopy.distance

In [ ]:
#defining distance function to calc each customer's distance to pickup
def distance(x,y):
    try:
        result = geopy.distance.geodesic(x,y).miles
    except ValueError:
        result = 0
    return result

In [ ]:
#running function
geocoding_df['Distance_to_pickup']=geocoding_df.apply(lambda x: distance((x.latitude, x.longitude),(x.Pickup_lat,x.Pickup_long)),axis=1)

In [ ]:
geocoding_df.head()

In [ ]:
new_df.head()

In [ ]:
geocoding_df.columns

In [ ]:
#Merging the distances to pickup back onto our main dataframe.
df = new_df.merge(geocoding_df[['IDMemb', 'latitude', 'longitude', 'Pickup_lat',
       'Pickup_long', 'Distance_to_pickup']], how = 'left', on = 'IDMemb')

In [ ]:
df.describe()

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
import datetime

In [ ]:
#Getting When registered as a dateteime object.
df['WhenReg'] = pd.to_datetime(df['WhenReg'])

In [ ]:
#Subtract when the member registered from the pickup day of the cycle to see how long they have been a member.
df['Years_member'] = (df['WhenStartPickup'] - df['WhenReg'])/datetime.timedelta(days=365)

In [ ]:
#This fixes the dataframe since currently it has cycle created for every member, even if they hadn't signed up yet. Now, we can delete all rows
#where the member hadn't registered yet.
df = df[df['Years_member']>0]

In [ ]:
df.columns

In [ ]:
cycles = df['IDCyc'].unique()
dates = df['WhenStartPickup'].unique()

In [ ]:
date_cycle_num_df=pd.DataFrame()
date_cycle_num_df['IDCyc'] = cycles
date_cycle_num_df['WhenStartPickup']=dates

In [ ]:
date_cycle_num_df.head()

In [ ]:
#Adding quarters to the data
date_cycle_num_df['quarter'] = pd.PeriodIndex(date_cycle_num_df.WhenStartPickup, freq='Q')

In [ ]:
#Adding months to the data
date_cycle_num_df['month'] = pd.DatetimeIndex(date_cycle_num_df['WhenStartPickup']).month

In [ ]:
date_cycle_num_df.head()

In [ ]:
#holiday column. Starting with all 0's, which means no holiday. 1 will be just before a major holiday, 2 will
#will mean just after a holiday. Only for holidays within 10 days of pickup. Chose to use the following holidays:
#New Years, Valentine's Day, Memorial Day, Independence Day, Labor Day, Thanksgiving, Christmas and New Year's
date_cycle_num_df['holiday'] = 0

In [ ]:
date_cycle_num_df.head()

In [ ]:
date_cycle_num_df.at[6,'holiday']=2 # after Valentines
date_cycle_num_df.at[7,'holiday']=1 # before Valentines
date_cycle_num_df.at[10,'holiday']=2 #after Christmas
date_cycle_num_df.at[11,'holiday']=1 #before Christmas
date_cycle_num_df.at[12,'holiday']=2 #after Thanksgiving
date_cycle_num_df.at[13,'holiday']=1 #before Thanksgiving
date_cycle_num_df.at[18,'holiday']=2 #after Labor Day
date_cycle_num_df.at[22,'holiday']=2 # after July 4
date_cycle_num_df.at[23,'holiday']=1 # before July 4
date_cycle_num_df.at[25,'holiday']=2 #after Memorial Day
date_cycle_num_df.at[29,'holiday']=1 # before Easter
date_cycle_num_df.at[28,'holiday']=2 #after Easter
date_cycle_num_df.at[32,'holiday']=2 # after Valentines
date_cycle_num_df.at[33,'holiday']=1 # before Valentines
date_cycle_num_df.at[36,'holiday']=2 #after Christmas
date_cycle_num_df.at[37,'holiday']=1 #before Christmas
date_cycle_num_df.at[38,'holiday']=2 #after Thanksgiving
date_cycle_num_df.at[39,'holiday']=1 #before Thanksgiving
date_cycle_num_df.at[44,'holiday']=2 #after Memorial Day
date_cycle_num_df.at[49,'holiday']=1 # before July 4

In [ ]:
date_cycle_num_df.head()

In [ ]:
#merging the holiday info onto the main dataframe
df = df.merge(date_cycle_num_df, how='left', on = ['IDCyc', 'WhenStartPickup'])

In [ ]:
df.head()

In [ ]:
filepath = '/Users/emilydanielbowser/Documents/Iowa Food Coop/Data/Intermediate Data/Wrangled_data'
df.to_csv(filepath)